In [ ]:
!pip install tensorflow opencv-python

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout #“These layers form the convolutional neural network used for facial expression recognition.”
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score
from google.colab import files


In [ ]:
from google.colab import files
uploaded = files.upload()  # select CK+48.zip

Saving CK+48.zip to CK+48.zip


In [ ]:
!rm -rf ./ckplus/CK+48
!unzip CK+48.zip -d ./ckplus


Archive:  CK+48.zip
   creating: ./ckplus/CK+48/
   creating: ./ckplus/CK+48/anger/
 extracting: ./ckplus/CK+48/anger/S010_004_00000017.png  
 extracting: ./ckplus/CK+48/anger/S010_004_00000018.png  
 extracting: ./ckplus/CK+48/anger/S010_004_00000019.png  
 extracting: ./ckplus/CK+48/anger/S011_004_00000019.png  
 extracting: ./ckplus/CK+48/anger/S011_004_00000020.png  
 extracting: ./ckplus/CK+48/anger/S011_004_00000021.png  
 extracting: ./ckplus/CK+48/anger/S014_003_00000028.png  
 extracting: ./ckplus/CK+48/anger/S014_003_00000029.png  
 extracting: ./ckplus/CK+48/anger/S014_003_00000030.png  
 extracting: ./ckplus/CK+48/anger/S022_005_00000030.png  
 extracting: ./ckplus/CK+48/anger/S022_005_00000031.png  
 extracting: ./ckplus/CK+48/anger/S022_005_00000032.png  
 extracting: ./ckplus/CK+48/anger/S026_003_00000013.png  
 extracting: ./ckplus/CK+48/anger/S026_003_00000014.png  
 extracting: ./ckplus/CK+48/anger/S026_003_00000015.png  
 extracting: ./ckplus/CK+48/anger/S028_001_000

In [ ]:
dataset_path = './ckplus/CK+48'
print(os.listdir(dataset_path))  # should show all 7 class folders


['fear', 'surprise', 'disgust', 'contempt', 'sadness', 'happy', 'anger']


In [ ]:
IMG_SIZE = 48
X = []
y = []

class_labels = sorted(os.listdir(dataset_path)) #read folde nam
label_map = {cls: idx for idx, cls in enumerate(class_labels)}#Converts class names → numeric labels


for cls in class_labels:
    cls_path = os.path.join(dataset_path, cls) # itrating each class and building path
    for img_file in os.listdir(cls_path): # reading each image
        img_path = os.path.join(cls_path, img_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # grayscale #to reduce complexit
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = cv2.equalizeHist(img)  # histogram equalization
        X.append(img)
        y.append(label_map[cls])

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)/255.0
y = tf.keras.utils.to_categorical(y, num_classes=len(class_labels))


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y.argmax(axis=1), random_state=42
)


In [ ]:
for cls in os.listdir(dataset_path):
    path = os.path.join(dataset_path, cls)
    print(cls, len(os.listdir(path)))


fear 75
surprise 249
disgust 177
contempt 54
sadness 84
happy 207
anger 135


In [ ]:
train_labels = np.argmax(y_train, axis=1)  # converting back one-hot vector into numeric index
class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights_array))
# Compute class weights so model can pay more attention to minority classe

In [ ]:
num_classes = y.shape[1]

model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE,1)),
    Conv2D(32,(3,3),activation='relu',padding='same'), # low level feature
    MaxPooling2D(2,2),

    Conv2D(64,(3,3),activation='relu',padding='same'),
    MaxPooling2D(2,2),

    Conv2D(128,(3,3),activation='relu',padding='same'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256,activation='relu'), # learn combination of features
    Dropout(0.5),
    Dense(num_classes,activation='softmax') # output o probablities
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy', metrics=['accuracy'])
# define 3 layer conolutional layer model  with fully connected layers

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    class_weight=class_weights,
    callbacks=[early_stop]
)


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 180ms/step - accuracy: 0.1155 - loss: 2.0162 - val_accuracy: 0.5025 - val_loss: 1.8921
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 223ms/step - accuracy: 0.2985 - loss: 1.8616 - val_accuracy: 0.5279 - val_loss: 1.7803
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 0.4189 - loss: 1.6805 - val_accuracy: 0.6244 - val_loss: 1.2757
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 232ms/step - accuracy: 0.6005 - loss: 1.3631 - val_accuracy: 0.7513 - val_loss: 0.8059
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.7363 - loss: 0.8930 - val_accuracy: 0.7766 - val_loss: 0.6945
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 177ms/step - accuracy: 0.7811 - loss: 0.7354 - val_accuracy: 0.8731 - val_loss: 0.4788
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 224ms/step - accuracy: 0.8614 - loss: 0.5341 - val_accuracy: 0.8528 - val_loss: 0.4385
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.9004 - loss: 0.4045 - val_accuracy: 0.

In [ ]:
print("Overall Accuracy:", accuracy*100, "%")
print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))


NameError: name 'accuracy' is not defined

In [ ]:
y_pred = np.argmax(model.predict(X_val), axis=1)
y_true = np.argmax(y_val, axis=1)

accuracy = accuracy_score(y_true, y_pred)
print("Overall Accuracy:", accuracy*100, "%")
print(classification_report(y_true, y_pred, target_names=class_labels))
#“We use the trained model to predict validation images, compare predictions with true labels, calculate overall accuracy, and show per-class metrics with precision, recall, and F1-score.”


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Overall Accuracy: 98.47715736040608 %
              precision    recall  f1-score   support

       anger       1.00      0.89      0.94        27
    contempt       1.00      1.00      1.00        11
     disgust       1.00      1.00      1.00        35
        fear       1.00      1.00      1.00        15
       happy       1.00      1.00      1.00        42
     sadness       0.85      1.00      0.92        17
    surprise       1.00      1.00      1.00        50

    accuracy                           0.98       197
   macro avg       0.98      0.98      0.98       197
weighted avg       0.99      0.98      0.98       197



In [ ]:
print("Overall Accuracy:", accuracy*100, "%")


Overall Accuracy: 98.47715736040608 %


In [ ]:
# 1️⃣ Define Focal Loss (class-balanced loss)
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon()) #prevents numerical errors
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) # force model to learn difficult classes
        loss = weight * cross_entropy # apply focal weight to loss
        return K.sum(loss, axis=1)
    return focal_loss_fixed  # total loss computation

# 2️⃣ Create CB-CCNN model (same architecture as baseline)
model_cb = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE,1)),
    Conv2D(32,(3,3),activation='relu',padding='same'),
    MaxPooling2D(2,2),

    Conv2D(64,(3,3),activation='relu',padding='same'),
    MaxPooling2D(2,2),

    Conv2D(128,(3,3),activation='relu',padding='same'),
    MaxPooling2D(2,2),

    Flatten(), # Converts feature maps into a single vector
    Dense(256,activation='relu'),
    Dropout(0.5),
    Dense(num_classes,activation='softmax')
])

# 3️⃣ Compile with Focal Loss
model_cb.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss=focal_loss(gamma=2., alpha=0.25), #key difference from baseline ,improve learning for minority classes
    metrics=['accuracy']
)

# 4️⃣ Early stopping
early_stop_cb = EarlyStopping(
    monitor='val_accuracy',
    patience=7,
    restore_best_weights=True
)

# 5️⃣ Train CB-CCNN with class weights
history_cb = model_cb.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    class_weight=class_weights,  # gives more importance to rare classes
    callbacks=[early_stop_cb]
)

# 6️⃣ Evaluate CB-CCNN
y_pred_cb = np.argmax(model_cb.predict(X_val), axis=1)
y_true_cb = np.argmax(y_val, axis=1)

from sklearn.metrics import classification_report, accuracy_score

accuracy_cb = accuracy_score(y_true_cb, y_pred_cb)
print("CB-CCNN Overall Accuracy:", accuracy_cb*100, "%")
print(classification_report(y_true_cb, y_pred_cb, target_names=class_labels))

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 214ms/step - accuracy: 0.1158 - loss: 0.3720 - val_accuracy: 0.1269 - val_loss: 0.3519
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 169ms/step - accuracy: 0.2880 - loss: 0.3493 - val_accuracy: 0.1980 - val_loss: 0.3461
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 221ms/step - accuracy: 0.3326 - loss: 0.3242 - val_accuracy: 0.6244 - val_loss: 0.2624
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.5938 - loss: 0.2607 - val_accuracy: 0.6599 - val_loss: 0.1429
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 193ms/step - accuracy: 0.6562 - loss: 0.1745 - val_accuracy: 0.7919 - val_loss: 0.0901
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 191ms/step - accuracy: 0.7516 - loss: 0.1117 - val_accuracy: 0.8579 - val_loss: 0.0670
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.8421 - loss: 0.0698 - val_accuracy: 0.8579 - val_loss: 0.0539
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 224ms/step - accuracy: 0.8519 - loss: 0.0522 - val_accuracy: 0.

In [ ]:
# ------------------------------
# Compare Baseline vs CB-CCNN
# ------------------------------
y_pred_base = np.argmax(model.predict(X_val), axis=1)
y_pred_cb = np.argmax(model_cb.predict(X_val), axis=1)
y_true = np.argmax(y_val, axis=1)

from sklearn.metrics import accuracy_score, f1_score, classification_report

print("===== Baseline CCNN =====")
print("Overall Accuracy:", accuracy_score(y_true, y_pred_base)*100)
print("Macro F1-score:", f1_score(y_true, y_pred_base, average='macro'))
report_base = classification_report(y_true, y_pred_base, target_names=class_labels, output_dict=True)
print("Per-class Recall:")
for cls in class_labels:
    print(f"  {cls}: {report_base[cls]['recall']:.2f}")

print("\n===== CB-CCNN =====")
print("Overall Accuracy:", accuracy_score(y_true, y_pred_cb)*100)
print("Macro F1-score:", f1_score(y_true, y_pred_cb, average='macro'))
report_cb = classification_report(y_true, y_pred_cb, target_names=class_labels, output_dict=True)
print("Per-class Recall:")
for cls in class_labels:
    print(f"  {cls}: {report_cb[cls]['recall']:.2f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
===== Baseline CCNN =====
Overall Accuracy: 100.0
Macro F1-score: 1.0
Per-class Recall:
  anger: 1.00
  contempt: 1.00
  disgust: 1.00
  fear: 1.00
  happy: 1.00
  sadness: 1.00
  surprise: 1.00

===== CB-CCNN =====
Overall Accuracy: 97.46192893401016
Macro F1-score: 0.9685832893670802
Per-class Recall:
  anger: 1.00
  contempt: 1.00
  disgust: 1.00
  fear: 0.87
  happy: 1.00
  sadness: 0.88
  surprise: 0.98
